In [1]:
import torch 
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

%cd ../

/workspace/Walk_Video_PyTorch/project


In [2]:
from models.pytorchvideo_models import WalkVideoClassificationLightningModule
from pytorch_lightning import Trainer

from IPython.display import clear_output

clear_output()

In [3]:
from pytorch_lightning import seed_everything

seed_everything(42, workers=True)

Global seed set to 42


42

In [4]:
from main import get_parameters

opt, unknown = get_parameters()
opt.num_workers = 8
opt.batch_size = 4
opt.gpu_num = 1

opt.version = 'test'
opt.model = "resnet"
opt.model_depth = 50

opt.clip_duarion = 3
opt.uniform_temporal_subsample_num = 64
opt.version = opt.version + '_' + opt.model + '_depth' + str(opt.model_depth)


In [5]:
from utils.utils import get_ckpt_path

model = WalkVideoClassificationLightningModule(opt)

# get last ckpt path
ckpt_path = get_ckpt_path(opt)

# model = WalkVideoClassificationLightningModule.load_from_checkpoint(ckpt_path)
# model = model.load_from_checkpoint(ckpt_path)

model.eval()

# clear_output()
print(ckpt_path)


FileNotFoundError: [Errno 2] No such file or directory: '/workspace/Walk_Video_PyTorch/logs/resnet/0811_resnet_depth50/checkpoints'

In [23]:
from dataloader.data_loader import WalkDataModule
from pytorch_lightning import loggers as pl_loggers 

# load test dataset 
module = WalkDataModule(opt)
module.setup()
test_data = module.test_dataloader()

# for the tensorboard
tb_logger = pl_loggers.TensorBoardLogger(save_dir="/workspace/Walk_Video_PyTorch/project/tests/logs", name=opt.model, version=opt.version)

trainer = Trainer(
    accelerator="auto",
    devices=1,
    gpus=opt.gpu_num,
    logger=tb_logger,
    max_epochs=1,
    # deterministic=True,
)

trainer.test(dataloaders=module, model=model, ckpt_path=ckpt_path)


/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:460: UserWarning: The flag `devices=1` will be ignored, instead the device specific number 1 will be used
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /workspace/Walk_Video_PyTorch/logs/resnet/0811_resnet_depth50/checkpoints/epoch=99-step=83200.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]
Loaded model weights from checkpoint at /workspace/Walk_Video_PyTorch/logs/resnet/0811_resnet_depth50/checkpoints/epoch=99-step=83200.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.2099267691373825     │
│      test_loss_epoch      │     6.170172214508057     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 6.170172214508057, 'test_acc_epoch': 0.2099267691373825}]

In [22]:
trainer.validate(dataloaders=module, model=model, ckpt_path=ckpt_path)

Restoring states from the checkpoint path at /workspace/Walk_Video_PyTorch/logs/resnet/0811_resnet_depth50/checkpoints/epoch=99-step=83200.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]
Loaded model weights from checkpoint at /workspace/Walk_Video_PyTorch/logs/resnet/0811_resnet_depth50/checkpoints/epoch=99-step=83200.ckpt


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_acc_epoch       │    0.21399511396884918    │
│      val_loss_epoch       │     6.216749668121338     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss_epoch': 6.216749668121338, 'val_acc_epoch': 0.21399511396884918}]

In [7]:
input_data = next(iter(test_data))


In [8]:
input_data['video'].shape

input_data["video_name"]

# input_data['video'][0][0][0][0]

['20200528_DHS_lat_V1-0036.mp4',
 '20200528_DHS_lat_V1-0036.mp4',
 '20200528_DHS_lat_V1-0036.mp4',
 '20200528_DHS_lat_V1-0036.mp4']

In [9]:
input_data['label']

tensor([1, 1, 1, 1])

In [10]:
preds = model(input_data['video'])

In [11]:
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=1).indices

In [12]:
pred_classes

tensor([[0],
        [0],
        [0],
        [0]])

In [13]:
from torch import softmax


pred_chagne = softmax(preds, dim=-1)
pred_chagne

tensor([[0.6938, 0.3062],
        [0.7311, 0.2689],
        [0.7299, 0.2701],
        [0.7311, 0.2689]], grad_fn=<SoftmaxBackward0>)

In [14]:
from utils.metrics import get_Accuracy

accuracy = get_Accuracy()

accuracy(pred_chagne, input_data['label'])

tensor(0.)

In [15]:
classname = {}

classname[0] = 'asd'
classname[1] = 'asd_not'

In [16]:
real_calss = []

for i in input_data['label'].tolist():
    real_calss.append(classname[i])

In [17]:
real_calss

['asd_not', 'asd_not', 'asd_not', 'asd_not']

In [18]:
# pred_class_names = []
# for num in range(opt._BATCH_SIZE):
#     for i in pred_classes[i]:
#         pred_class_names.append(classname[int(i)])


pred_class_names = [classname[int(i)] for i in pred_classes]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print("real label: %s" % ",".join(real_calss))

Predicted labels: asd, asd, asd, asd
real label: asd_not,asd_not,asd_not,asd_not


In [19]:
pred_class_names == real_calss

False

In [20]:
result = []

for i in range(len(real_calss)):
    if pred_class_names[i] == real_calss[i]:
        result.append("true")
    else:
        result.append("false")

result

['false', 'false', 'false', 'false']